# Count $<i,j>$ pairs as inputs to learn Node Embedding

In [27]:
%matplotlib inline
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from scipy.io import loadmat
from random import choices
import copy
from scipy.sparse import issparse
from statistics import mean,stdev

## Find percolation threshold. Avarage size of largest cluster $<s_{max}>$ v.s. $\beta$

### 1. Transform edgelist to Graph

In [28]:
def load_matfile(file_, variable_name="network", unDirected=True, unWeighted = True):
    mat_variables = loadmat(file_)
    mat_matrix = mat_variables[variable_name]
    if issparse(mat_matrix):
        if unDirected:
            G = nx.Graph()
        else:
            G = nx.DiGraph()
        '''
        from_nodes: cx.row
        to_nodes: cx.col
        link weights: cx.data
        '''
        cx = mat_matrix.tocoo()
        if unWeighted:
            edge_list = np.array([cx.row, cx.col]).T
            G.add_edges_from(edge_list)
        else:
            edge_list = np.array([cx.row, cx.col, cx.data]).T
            G.add_weighted_edges_from(edge_list)
    return G

In [29]:
# Here we don't consider Directed/Weighted graph 
def percolate(G, beta, unWeighted = True):
    edges_percolated = []
    G_percolated = nx.Graph()
    assert(G_percolated.number_of_nodes() == 0)
    numPercolatedEdges = int(beta*G.number_of_edges())
    G_percolated.add_nodes_from(list(G.nodes))
    if unWeighted:
        edges_percolated = choices(list(G.edges), k=numPercolatedEdges)
        assert(len(edges_percolated) ==  numPercolatedEdges)
        G_percolated.add_edges_from(edges_percolated)
#     else:
#         bond percolation probability depends on the edge's weight
    return G_percolated

In [30]:
def cal_average_Gc_sizes(G, betas, numIter, logging=False):
    average_sizes_Gc = []
    for beta in betas:
        sizes_Gc = []
        for i in range(numIter):
            G_percolated = percolate(G, beta)
            # graphs = list(nx.connected_component_subgraphs(G))
            # largest percolated cluster
            Gc = max(nx.connected_component_subgraphs(G_percolated), key=len)
            sizes_Gc.append(Gc.number_of_nodes())
        average_sizes_Gc.append([mean(sizes_Gc),stdev(sizes_Gc)])
        if logging:
            print("beta = {0:.4f} calculated".format(beta))
    return average_sizes_Gc

In [ ]:
if __name__ == "__main__":
    # "blogcatalog.mat" is a undirected, unweighted graph
    G_blogcatalog = load_matfile(file_ = "blogcatalog.mat")
    assert(G_blogcatalog.number_of_nodes() == 10312)
    assert(G_blogcatalog.number_of_edges() == 333983)
    
    # percolate
    betas = np.linspace(0,0.5,50)
    numIter = 2
    aver_sizes_Gc = cal_average_Gc_sizes(G = G_blogcatalog, 
                                         betas = betas, 
                                         numIter = numIter,
                                         logging = True)
    
    # plot 
    data = np.array(aver_sizes_Gc).T
    means = data[0,:]
    stdev = data[1,:]
    plt.errorbar(betas, means, yerr=stdev, fmt='o')
    plt.title('max(Gc_size)-beta')